In [1]:
import transformers
import json
import os
import torch
transformers.__version__

'4.16.0'

### Do some explore for the data

In [2]:
import pandas as pd

file_name = 'category_descriptions.csv'

df = pd.read_csv(file_name)

df.head()

,Category (incl. context and answer),Description,Answer Format,Group
0,Category: Document Name,Description: The name of the contract,Answer Format: Contract Name,Group: -
1,Category: Parties,Description: The two or more parties who signe...,Answer Format: Entity or individual names,Group: -
2,Category: Agreement Date,Description: The date of the contract,Answer Format: Date (mm/dd/yyyy),Group: 1
3,Category: Effective Date,Description: The date when the contract is eff...,Answer Format: Date (mm/dd/yyyy),Group: 1
4,Category: Expiration Date,Description: On what date will the contract's ...,Answer Format: Date (mm/dd/yyyy) / Perpetual,Group: 1


In [3]:
# what data type contains.
# this is all the supported clauses
types = [x.split(':')[-1].strip() for x in df['Category (incl. context and answer)'].values.tolist()]
print(types)

['Document Name', 'Parties', 'Agreement Date', 'Effective Date', 'Expiration Date', 'Renewal Term', 'Notice Period to Terminate Renewal', 'Governing Law', 'Most Favored Nation', 'Non-Compete', 'Exclusivity', 'No-Solicit of Customers', 'Competitive Restriction Exception', 'No-Solicit of Employees', 'Non-Disparagement', 'Termination for Convenience', 'Rofr/Rofo/Rofn', 'Change of Control', 'Anti-Assignment', 'Revenue/Profit Sharing', 'Price Restrictions', 'Minimum Commitment', 'Volume Restriction', 'IP Ownership Assignment', 'Joint IP Ownership', 'License Grant', 'Non-Transferable License', 'Affiliate License-Licensor', 'Affiliate License-Licensee', 'Unlimited/All-You-Can-Eat-License', 'Irrevocable or Perpetual License', 'Source Code Escrow', 'Post-Termination Services', 'Audit Rights', 'Uncapped Liability', 'Cap on Liability', 'Liquidated Damages', 'Warranty Duration', 'Insurance', 'Covenant Not to Sue', 'Third Party Beneficiary']


In [4]:
import json

data = json.load(open('CUADv1.json'))

len(data)

2

In [5]:
data['data'][0].keys()

dict_keys(['title', 'paragraphs'])

In [6]:
# there are 510 pdfs
len(data['data'])

510

In [17]:
model_id ="distilbert-base-cased-distilled-squad"

from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForQuestionAnswering.from_pretrained(model_id)

HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /distilbert-base-cased-distilled-squad/resolve/main/config.json (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1129)')))


OSError: Can't load config for 'distilbert-base-cased-distilled-squad'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'distilbert-base-cased-distilled-squad' is the correct path to a directory containing a config.json file

In [7]:
import json
def load_json(file_name):
    with open(file_name, 'r') as f:
        data = json.loads(f.read())
    return data


train_data = load_json('sample.json')

len(train_data)

2

```json
{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}```

In [14]:
# let's create a func to make the real data with 
import copy

train_data = load_json('sample.json')

def get_trained_data(train_data):
  real_train_ds = []
  for i, d in enumerate(train_data['data']):
      ps = d['paragraphs']
      for p in ps:
          context = p['context']
          for qas in p['qas']:
              qas['context'] = context
              qas.pop('is_impossible', '')
              if qas['answers'] == 0:
                  continue
              if len(qas['answers'])  >= 1:
                  # print(qas['answers'])
                  tmp_ans = qas['answers'][0]
                  tmp_ans['text'] = [tmp_ans['text']]
                  tmp_ans['answer_start'] = [tmp_ans['answer_start']]
                  qas['answers'] = tmp_ans
              
              # qas['answers']['text'] = [qas['answers']['text']]     
              real_train_ds.append(qas) 

  real_train_ds = [x for x in real_train_ds if x['answers'] != []]
  return real_train_ds

real_train_ds = get_trained_data(train_data)

len(real_train_ds)

132

In [16]:
import pandas as pd
from datasets import Dataset
# filter with answer will null
# new_ds = [x for x in real_train_ds if x['answers'] != []]

# len(new_ds)

df = pd.DataFrame(real_train_ds)

dataset = Dataset.from_pandas(df)
print(len(dataset))

132


In [10]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

train_ds_new = dataset.map(preprocess_training_examples, batched=True, remove_columns=dataset.column_names)



[{'answers': [{'text': 'DISTRIBUTOR AGREEMENT', 'answer_start': 44}],
  'id': 'LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT__Document Name',
  'question': 'Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract',
  'is_impossible': False},
 {'answers': [{'text': 'Distributor', 'answer_start': 244},
   {'text': 'Electric City Corp.', 'answer_start': 148},
   {'text': 'Electric City of Illinois L.L.C.', 'answer_start': 49574},
   {'text': 'Company', 'answer_start': 197},
   {'text': 'Electric City of Illinois LLC', 'answer_start': 212}],
  'id': 'LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT__Parties',
  'question': 'Highlight the parts (if any) of this contract related to "Parties" that should be reviewed by a lawyer. Details: The two or more parties who signed the contract',
  'is_impossible': False},
 {'answers': [{'text': '7th day of September, 1999.', 'answer_start': 263}],
  'id': 

In [11]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-squad",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds_new,
    eval_dataset=train_ds_new,
    tokenizer=tokenizer,
)
trainer.train()

82

In [65]:
from torch.utils.data import DataLoader, Dataset


class MyData(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        data = self.data[i]
        return preprocess_training_examples(data)
    
dataset = MyData(real_train_ds)

loader = DataLoader(dataset, batch_size=2, shuffle=True)
    
    


In [36]:
train_data['data'][0]['paragraphs'][0]['qas'][0]

{'answers': [{'text': 'DISTRIBUTOR AGREEMENT', 'answer_start': 44}],
 'id': 'LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT__Document Name',
 'question': 'Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract',
 'is_impossible': False}